### **Análise dos dados**

Segue abaixo a análise dos dados inicialmente fornecidos.

Imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Inicializando os dados

In [ ]:
df_resultados = pd.read_csv('data/RESULTADOS_04_06_2024_full_teste.csv', compression='gzip')
df_falhas = pd.read_csv('data/FALHAS_04_05_06_2024.csv')

Checando os dados

In [ ]:
# Remover coluna errada
df_resultados = df_resultados.drop(columns=['Unnamed: 0'], axis=1)
df_falhas = df_falhas.drop(columns=['Unnamed: 0'], axis=1)


Arrumar os dados de falha

In [ ]:
df_falhas.columns = df_falhas.iloc[0]

df_falhas = df_falhas.drop(df_falhas.index[0])

Checar ambos DFs

In [ ]:
df_falhas.head(20)

In [ ]:
df_resultados.head(20)
print("m700:", 'M700' in df_resultados['STATUS'].values)

Analisando os tipos de dados de falhas

In [ ]:
df_falhas.describe()

In [ ]:
df_resultados.head(20)

Após está analise, creio que os dados de ambas as tabelas que não serão utilizados tanto no treinamento quanto teste do modelo são:

#### Falhas:

- KNR
- Estação
- Data

#### Resultados:

- KNR
- Name
- Data

OBS(Futuramente a coluna Data pode ser utilizada para criar novas features)
OBS2(Colunas KNR e Name serão removidas apenas antes do treinamento do modelo)

## Preparação dos dados

Inicialmente, os dados serão integrados em uma única tabela

In [ ]:
# Remoção das colunas com NaN

df_falhas = df_falhas.dropna()
df_resultados = df_resultados.dropna()

In [ ]:
# Randomly sample 50% of the DataFrame
df_resultados_reduced = df_resultados.sample(frac=0.5, random_state=42)


In [ ]:
# Modificando o nome da tabela DATA em cada dataframe

df_falhas = df_falhas.rename(columns={'DATA': 'DATA_FALHAS'})
df_resultados_reduced = df_resultados_reduced.rename(columns={'DATA': 'DATA_RESULTADOS'})

# Juntando os dois dataframes

merged_df = pd.merge(df_resultados_reduced, df_falhas, on='KNR', how='inner')

In [ ]:
df_resultados.shape

In [ ]:
df_falhas.shape

In [ ]:
merged_df.head(10)

In [ ]:
merged_df.to_csv('data/merged_df.csv', compression='gzip')

In [ ]:
falhas_aggregated = df_falhas.groupby('KNR').agg({
    'MODELO': 'first',
    'MOTOR': 'first',
    'COR': 'first',
    'HALLE': lambda x: '; '.join(x.astype(str)),  # Concatenando os HALLES
    'FALHA': lambda x: '; '.join(x.astype(str)),   # Concatenando as FALHAS
    'DATA_FALHAS': 'first'
}).reset_index()

# Agrupando e concatenando os dados da tabela de resultados
resultados_aggregated = df_resultados.groupby('KNR').agg({
    'NAME': lambda x: '; '.join(x.astype(str)),     # Concatenando as NAMES
    'ID': lambda x: '; '.join(x.astype(str)),       # Concatenando os IDs
    'STATUS': lambda x: '; '.join(x.astype(str)),   # Concatenando os STATUS
    'UNIT': lambda x: '; '.join(x.dropna().astype(str)), # Concatenando as UNIT
    'VALUE_ID': lambda x: '; '.join(x.dropna().astype(str)), # Concatenando os VALUE_IDs
    'VALUE': lambda x: '; '.join(x.dropna().astype(str)),     # Concatenando os VALUES
    'DATA': 'first'
}).reset_index()


In [ ]:
resultados_aggregated.head(30)

In [ ]:

resultados_aggregated = resultados_aggregated.rename(columns={'DATA': 'DATA_RESULTADOS'})

# Juntando os dois dataframes

merged_df = pd.merge(resultados_aggregated, falhas_aggregated, on='KNR', how='inner')

In [ ]:
merged_df.head(10)